# LangChain: Memory

## Outline
* ConversationBufferMemory
* ConversationBufferWidowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory
* (Vector data memory)
* (Entity memory): Facts related to entities. 

## ConversationBufferMemory
$\Rightarrow$ LLM is stateless, _i.e._ it does not store any information about the conversation history. Each transaction is indep. We need memory to store the conversation history. We use a simple memory buffer to store the convo. The memory storage is used as input for future prediction.     
$\Rightarrow$ Langchain provides numerous ways for the memory. 

In [50]:
# pip install tiktoken

In [51]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [52]:
import langchain
print(langchain.__version__)

0.0.313


In [53]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [54]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True # shows the memory when making a prediction 
)

In [55]:
conversation.predict(input="Hi, my name is Andrew") # langchain will create a prompt for the convo



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:



> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [56]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 is equal to 2.'

In [57]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew.'

In [58]:
print(memory.buffer) # saved convo 

Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI: Your name is Andrew.


In [59]:
memory.load_memory_variables({}) # langchain memory 

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 is equal to 2.\nHuman: What is my name?\nAI: Your name is Andrew."}

In [66]:
memory = ConversationBufferMemory()

In [67]:
print('Variables:\n', memory.load_memory_variables({}), sep='')
print('\nBuffer:\n', memory.buffer, sep='')

Variables:
{'history': ''}

Buffer:



In [68]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [69]:
print('Variables:\n', memory.load_memory_variables({}), sep='')
print('\nBuffer:\n', memory.buffer, sep='')

Variables:
{'history': "Human: Hi\nAI: What's up"}

Buffer:
Human: Hi
AI: What's up


In [70]:
memory.save_context({"input": "Not much, just hanging"}, # append to memory  s
                    {"output": "Cool"})

In [71]:
print('Variables:\n', memory.load_memory_variables({}), sep='')
print('\nBuffer:\n', memory.buffer, sep='')

Variables:
{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

Buffer:
Human: Hi
AI: What's up
Human: Not much, just hanging
AI: Cool


## ConversationBufferWindowMemory
Only keeps a windows of the memory 

In [15]:
from langchain.memory import ConversationBufferWindowMemory

In [75]:
memory = ConversationBufferWindowMemory(k=1) # windows size of 1, # of convo to memorize            

In [76]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [77]:
print('Variables:\n', memory.load_memory_variables({}), sep='')
print('\nBuffer:\n', memory.buffer, sep='')

Variables:
{'history': 'Human: Not much, just hanging\nAI: Cool'}

Buffer:
Human: Not much, just hanging
AI: Cool


In [78]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [79]:
conversation.predict(input="Hi, my name is Andrew")

"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [80]:
conversation.predict(input="What is 1+1?")

'1+1 is equal to 2.'

In [81]:
conversation.predict(input="What is my name?") # forget the convo for k > 1 distance. It now only remembers the '1+1' convo 

"I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation."

## ConversationTokenBufferMemory
Limit the # of token saved (as the cost depends on the # of tokens)

In [92]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0)

In [99]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=100) # specify llm as different llm count the token differently 
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [100]:
print('Variables:\n', memory.load_memory_variables({}), sep='')
print('\nBuffer:\n', memory.buffer, sep='')

Variables:
{'history': 'Human: AI is what?!\nAI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

Buffer:
Human: AI is what?!
AI: Amazing!
Human: Backpropagation is what?
AI: Beautiful!
Human: Chatbots are what?
AI: Charming!


In [97]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30) 
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [98]:
print('Variables:\n', memory.load_memory_variables({}), sep='')
print('\nBuffer:\n', memory.buffer, sep='')

Variables:
{'history': 'AI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

Buffer:
AI: Beautiful!
Human: Chatbots are what?
AI: Charming!


## ConversationSummaryMemory

#### For limited token in the memory. A summary will be automatically created by the OpenAI API. 
The history needs to comply with the total token limit.

In [102]:
from langchain.memory import ConversationSummaryBufferMemory

In [131]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100) # if sufficient token, it will summarize the convo
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [132]:
print('Variables:\n', memory.load_memory_variables({}), sep='')
print('\nBuffer:\n', memory.buffer, sep='')

Variables:
{'history': 'System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting.'}

Buffer:
[]


In [133]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [134]:
conversation.predict(input="What would be a good demo to show?") # THE "System" is automatically generated by OpenAi. 



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting.
Human: What would be a good demo to show?
AI:

> Finished chain.


'A good demo to show during the lunch meeting with the customer interested in AI would be the latest LLM (Language Model) demo. The LLM is a cutting-edge AI model that can generate human-like text based on a given prompt. It has been trained on a vast amount of data and can generate coherent and contextually relevant responses. By showcasing the LLM demo, you can demonstrate the capabilities of AI in natural language processing and how it can be applied to various industries and use cases.'

The results including "system" will be stored in the memory. Since "system" summarizes the conversation history, it is not necessary to store the history to save tokens. 

In [135]:
print('Variables:\n', memory.load_memory_variables({}), sep='')  
print('\nBuffer:\n', memory.buffer, sep='')

Variables:
{'history': 'System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting. The human asks what would be a good demo to show, and the AI suggests showcasing the latest LLM (Language Model) demo. The LLM is a cutting-edge AI model that can generate human-like text based on a given prompt. It has been trained on a vast amount of data and can generate coherent and contextually relevant responses. By showcasing the LLM demo, you can demonstrate the capabilities of AI in natural language processing and how it can be applied to various industries and use cases.'}

Buffer:
[]


#### With sufficient token allowance, the whole conversation will be saved in the memory. No "system" will be generated by the OpenAI API.

In [126]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=600) # if sufficient token, it will save the convo 
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [127]:
print('Variables:\n', memory.load_memory_variables({}), sep='')
print('\nBuffer:\n', memory.buffer, sep='')

Variables:
{'history': "Human: Hello\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool\nHuman: What is on the schedule today?\nAI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo."}

Buffer:
[HumanMessage(content='Hello'), AIMessage(content="What's up"), HumanMessage(content='Not much, just hanging'), AIMessage(content='Cool'), HumanMessage(content='What is on the schedule today?'), AIMessage(content='There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. 

$\Rightarrow$ Now create a convo with the full context, no summary. 

In [128]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [129]:
conversation.predict(input="What would be a good demo to show?") 



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello
AI: What's up
Human: Not much, just hanging
AI: Cool
Human: What is on the schedule today?
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.
Human: What would be a good demo to show?
AI:

> Finished chain.


'A good demo to show would be the Language Model API, which can generate realistic and coherent text based on a given prompt. You can showcase its capabilities by providing a few example prompts and letting the API generate creative and informative responses. This will impress the customer and demonstrate the power of our AI technology.'

In [130]:
print('Variables:\n', memory.load_memory_variables({}), sep='') 
print('\nBuffer:\n', memory.buffer, sep='')

Variables:
{'history': "Human: Hello\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool\nHuman: What is on the schedule today?\nAI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.\nHuman: What would be a good demo to show?\nAI: A good demo to show would be the Language Model API, which can generate realistic and coherent text based on a given prompt. You can showcase its capabilities by providing a few example prompts and letting the API generate creative and informative responses. This will impress the customer and demonstrate the power of our AI technology."}

Buffer:
[HumanMessage(content='Hello'), AIMessage(content